In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

In [2]:
#reading in the csv file
staten_island_df = pd.read_csv("Resources/NYCCrimeData_Parks_Staten_Island_TableToExcel.csv")
staten_island_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OBJECTID      390 non-null    int64  
 1   Complaint ID  366 non-null    object 
 2   Offense       366 non-null    object 
 3   Latitude      390 non-null    float64
 4   Longitude     390 non-null    float64
 5   Borough       24 non-null     object 
 6   Public Area   24 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 21.5+ KB


In [3]:
#dividing the df into crimes and public institutions
r_crimes = staten_island_df.iloc[0:365, :]
r_crimes = r_crimes.drop(columns= ["Borough", "Public Area"])

In [4]:
r_crimes.head()

,OBJECTID,Complaint ID,Offense,Latitude,Longitude
0,1,261214832,ROBBERY,40.643566,-74.074001
1,2,261233859,ROBBERY,40.635961,-74.161365
2,3,261233841,ROBBERY,40.643569,-74.098338
3,4,261261343,ROBBERY,40.630408,-74.142977
4,5,261347872,ROBBERY,40.630539,-74.076641


In [5]:
#same method for public institutions and reordering columns
r_public = staten_island_df.iloc[366:389, :]
r_public = r_public.drop(columns= ["Complaint ID", "Offense"])
r_public = r_public[["OBJECTID", "Public Area", "Latitude", "Longitude"]]

In [6]:
r_public.head()

,OBJECTID,Public Area,Latitude,Longitude
366,367,PARKS,40.614533,-74.074185
367,368,RECREATION AND WATERFRONT SITES,40.510392,-74.230405
368,369,PUBLIC LIBRARIES,40.595802,-74.063027
369,370,PUBLIC LIBRARIES,40.590395,-74.101138
370,371,PARKS,40.638748,-74.083899


In [7]:
#each location a crime took place

r_crimes_gdf = gpd.GeoDataFrame(
    r_crimes,
    geometry=gpd.points_from_xy(
        r_crimes["Longitude"],
        r_crimes["Latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

# each public institution

r_public_gdf = gpd.GeoDataFrame(
    r_public, 
    geometry=gpd.points_from_xy(
        r_public["Longitude"],
        r_public["Latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

# EPSG:3857 converts it to meters

r_crimes_gdf_proj = r_crimes_gdf.to_crs({"init": "EPSG:3857"})
r_public_gdf_proj = r_public_gdf.to_crs({"init": "EPSG:3857"})

# 1 miles would be 1609 meters

x = r_public_gdf_proj.buffer(1609).unary_union

crimes_per = r_crimes_gdf_proj["geometry"].intersection(x)

# print all the nearby points
test = (r_crimes_gdf[~crimes_per.is_empty])
test

c:\Users\54qb3\anaconda3\envs\dev\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\54qb3\anaconda3\envs\dev\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\54qb3\anaconda3\envs\dev\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. Wh

,OBJECTID,Complaint ID,Offense,Latitude,Longitude,geometry
0,1,261214832,ROBBERY,40.643566,-74.074001,POINT (-74.07400 40.64357)
1,2,261233859,ROBBERY,40.635961,-74.161365,POINT (-74.16137 40.63596)
4,5,261347872,ROBBERY,40.630539,-74.076641,POINT (-74.07664 40.63054)
5,6,261434927,ROBBERY,40.637135,-74.123473,POINT (-74.12347 40.63714)
6,7,261431397,ROBBERY,40.643996,-74.076502,POINT (-74.07650 40.64400)
...,...,...,...,...,...,...
359,360,279208483,ROBBERY,40.644719,-74.081327,POINT (-74.08133 40.64472)
360,361,279411279,ROBBERY,40.635738,-74.132124,POINT (-74.13212 40.63574)
362,363,279447847,ROBBERY,40.632740,-74.152082,POINT (-74.15208 40.63274)
363,364,279472819,ROBBERY,40.614616,-74.084716,POINT (-74.08472 40.61462)
